In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
# DF = returnRawDFperBenchmark(Benchmark = "is", fix="Classes", benchmarkClass = ["A", "B", "C", "D"], FixedProcess = 64, Processes = [1,2,4,8,16,32,64,128,256], FixedBenchmarkClass="B")
# DF

In [ ]:
# raw_x = DF.columns.tolist()
# functionName = "MPI_Irecv()"
# raw_y = [DF.at[functionName, x] for x in raw_x]

# # グラフのプロット
# fig = plt.figure(figsize=(12, 9))
# ## 実データをsklearn用に変形
# raw_x = np.array(raw_x).reshape(-1,1)
# raw_y = np.array(raw_y).reshape(-1, 1)
# ## 説明変数と目的変数に分割
# notTrain_x, train_x, target_x = raw_x[0], raw_x[1:-1], raw_x[-1]
# notTrain_y, train_y, target_y = raw_y[0], raw_y[1:-1], raw_y[-1]
# ## 実データをそれぞれプロット
# plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
# plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")
# plt.scatter(notTrain_x, notTrain_y, marker="o", label="最初のデータを除外した時に予測に用いなかった関数コール回数")
# ## モデル式をプロットするために変数”plot_x”を用意する
# plot_x_min = 1.0
# plot_x_max = 256.0
# ### モデル式としてプロットする最低値の設定
# plot_x = np.linspace(plot_x_min, plot_x_max, 500)
# plot_x = np.array(plot_x).reshape(-1, 1)
# ## モデルを [1 - 1/n] として計算
# plot_y = 1 - 1/plot_x
# plt.plot(plot_x, plot_y)


# modelIpMk2 = ModelIp_mk2(train_x=train_x, train_y=train_y, target_x=target_x, target_y=target_y)
# modelIpMk2.calc_lr()
# plot_y = modelIpMk2.predict(plot_x)
# plt.plot(plot_x, plot_y)


In [ ]:
# raw_x

# raw_y

In [ ]:
benchmarkName = "cg"
fixed = "Process"
benchmarkClasses = ["A", "B", "C", "D"]
FixedProcess = 64
Processes = [1,2,4,8,16,32,64,128,256]
FixedBenchmarkClass = "B"

In [ ]:
DF = collectFunctionNamesPerBenchmark(benchmarkName = "cg", benchmarkClasses = benchmarkClasses, processes = Processes)
# for benchmarkClass in benchmarkClasses:
#     for Process in Processes:
#         pass

In [ ]:
DF

In [ ]:
functionNames = DF["functionName"]
functionNames = tuple(functionNames)
functionNames

In [ ]:
print(DF.query('functionName.str.endswith("VECSET")', engine='python'))